# Step 1: Setting Up the Python Application

In [1]:
!pip install langchain_openai langchain_core langchain_community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 502.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

### Import the required libraries

In [2]:
from langchain.memory import ChatMessageHistory

from langchain_openai import ChatOpenAI

from langchain_core.prompts import (ChatPromptTemplate,
                                    PromptTemplate
                                    )

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import (BaseModel,
                                        Field)

from langchain_community.callbacks import get_openai_callback


import chromadb
import chromadb.utils.embedding_functions as embedding_functions

from typing import List

from getpass import getpass

import csv
import textwrap


In [3]:
API_KEY  = getpass()

··········


In [4]:
CSV_FILE = 'HomeListing.csv'
MODEL_NAME = "gpt-3.5-turbo-1106"
TEMPERATURE_ONE = 1
TEMPERATURE_ZERO = 0
MAX_TOKENS = 4000
NUMBER_OF_LISTINGS = 10 # default value
TEXT_EMBEDDING_MODEL ='text-embedding-3-large'


class style:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [5]:
# define the class/basemodel to read in the JSON generated by the LLM

class singleHomeList (BaseModel):

  neighborhood : str = Field(description='name of the neighborhood')
  price        : str = Field(description='price of the home in US dollars')
  bedrooms     : int = Field(description='number of bedrooms in the house')
  bathrooms    : int = Field(description='number of bathrooms in the house')
  houseSize    : str = Field(description='size of the house in square feet')
  description  : str = Field(description='detailed features description of the listing')
  neighborhood_description: str = Field(description='detailed description of the neighborhood')


class HomeListings (BaseModel):
  Listings : List[singleHomeList]  = Field(description='list of real estate listings generated by the AI defined as singleHomeList model')

In [6]:
HomeListings.schema()

{'title': 'HomeListings',
 'type': 'object',
 'properties': {'Listings': {'title': 'Listings',
   'description': 'list of real estate listings generated by the AI defined as singleHomeList model',
   'type': 'array',
   'items': {'$ref': '#/definitions/singleHomeList'}}},
 'required': ['Listings'],
 'definitions': {'singleHomeList': {'title': 'singleHomeList',
   'type': 'object',
   'properties': {'neighborhood': {'title': 'Neighborhood',
     'description': 'name of the neighborhood',
     'type': 'string'},
    'price': {'title': 'Price',
     'description': 'price of the home in US dollars',
     'type': 'string'},
    'bedrooms': {'title': 'Bedrooms',
     'description': 'number of bedrooms in the house',
     'type': 'integer'},
    'bathrooms': {'title': 'Bathrooms',
     'description': 'number of bathrooms in the house',
     'type': 'integer'},
    'houseSize': {'title': 'Housesize',
     'description': 'size of the house in square feet',
     'type': 'string'},
    'descripti

In [7]:
# Instantiate Model
llm = ChatOpenAI(
    openai_api_key = API_KEY,
    temperature= TEMPERATURE_ONE,
    model= MODEL_NAME,
    max_tokens=MAX_TOKENS
)

# create the parsers for LLM response format
parser_homeListings = PydanticOutputParser(pydantic_object=HomeListings)
parser_singleListing = PydanticOutputParser(pydantic_object=singleHomeList)

format_instructions_homeListings  = parser_homeListings.get_format_instructions()
format_instructions_singleListing = parser_singleListing.get_format_instructions()


In [8]:
def get_chain(llm, prompt, parser):
  """

  Args:
    llm: LLM object for processing the prompts
    prompt: Prompt object for generating the response
    parser: Output parser object for parsing the response

  Returns:
    chain: Chain object for processing the prompts

  """
  if parser is None:
    return prompt | llm
  else:
    return prompt | llm | parser

In [9]:
def show_listing(listing : singleHomeList):
  """

  Args:
    listing: singleHomeList object for showing the listing

  Returns:
    None
  """
  formatStyle = style.BOLD+style.BLUE

  print("{}Neighborhood: {} {}".format(formatStyle, style.END, listing.neighborhood ))
  print("{}Price:{} {}".format(formatStyle, style.END, listing.price))
  print("{}Bedroom:{} {}".format(formatStyle, style.END, listing.bedrooms))
  print("{}Bathrooms:{} {}".format(formatStyle, style.END, listing.bathrooms))
  print("{}House Size:{} {}".format(formatStyle, style.END, listing.houseSize ))
  print(textwrap.fill("{}Description:{} {}".format(formatStyle, style.END, listing.description), 100))
  # print("\n")
  print(textwrap.fill("{}Neighborhood Description:{} {}".format(formatStyle, style.END, listing.neighborhood_description), 100))
  print("\n")

In [10]:
def get_vector_db_client(path):
  """

  Args:
    path: path to store the vector db

  Returns:
    client: vector db client

  """
  return chromadb.PersistentClient(path=path)

def get_embedding_function():
  """

  Returns:
    embedding_functions: embedding function object used to create the embedding

  """
  return embedding_functions.OpenAIEmbeddingFunction(
    api_key= API_KEY,
    model_name = TEXT_EMBEDDING_MODEL
  )

def get_collection(client, name, embedding_functions):
  """
  Args:
    client: vector db client
    name: name of the collection
    embedding_functions: embedding function object used to create the embedding, when not defined, default is used

  Returns:
    collection: collection object that will be used to store/retrieve the listing

  """
  if embedding_functions is None:
    return client.get_or_create_collection(
      name=name,
      metadata = {'hhsw:space' : 'cosine'}
    )
  else:
    return client.get_or_create_collection(
      name=name,
      embedding_function=embedding_functions,
      metadata = {'hhsw:space' : 'cosine'}
    )


# Step 2: Generating Real Estate Listings

### Setup the prompt to generate the real-estate listings

In [26]:
instructions = """You are a helpful AI with an expertise in real estate tasked with creating list of real estate listings for variety of homes in different neighborhoods.

INSTRUCTIONS THAT MUST BE STRICTLY FOLLOWED:
The reponse must include a neighborhood name, price in US dollars,
number of bedrooms should be anywhere between 2 and 6, number of bathrooms should be between 1 and
number of bedrooms, house size is in square feet and should be propotional to the number of
bedrooms.
Generate very detailed description about the listing and include its price, number of bedrooms, number of bathrooms,
House size,  energy source, flooring, appliances, type of structure, number of floors, describe dining room, family room,
kitchen, bathrooms, washer, dryers,  architectural style, construction material, year built, foundation, roof,
source of heating  cooling the home, does the listing includes a finished basement, unfinished basement or there is
no basement. Does it has a back yard, and a front yard. does property includes car garage. there is town sewerage or its
in-ground sewerage.
For neighborhood description, generate very detailed description that describes the community, area schools,
parks, promximty to highways, proximity to shopping malls, grocery, restaurants, lakes, rivers, open spaces.

================= START OF THE LISTING EXAMPLE ==================
{listing_example}
================= END OF THE LISTING EXAMPLE ==================

ONLY USE THE REQUESTED FORMAT.
================= START OF THE FORMATTING INSTRUCTIONS ==================
{output_format_instructions}
================= END OF THE FORMATTING INSTRUCTIONS ==================

"""

listing_example = [
"""
  Neighborhood: Green Oaks
  Price': $ 800,000
  Bedrooms: 3
  Bathrooms: 2
  House Size: 2,000 sqft
  Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 2,000 sqft, 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.
  Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
""",
"""
  Neighborhood: Woodland Hills
  Price': $ 950,000
  Bedrooms: 3
  Bathrooms: 3
  House Size: 2,300 sqft
  Description: Escape to this charming 3-bedroom, 3-bathroom retreat nestled in the serene neighborhood of Woodland Hills. The 2,300 sqft home features a light-filled open floor plan, a private backyard sanctuary, and an array of modern amenities. Enjoy the perfect blend of comfort and style in this delightful Woodland Hills residence.
  Neighborhood Description: Woodland Hills is a picturesque neighborhood known for its lush greenery, serene atmosphere, and proximity to hiking trails and recreational parks. Residents appreciate the peaceful surroundings and convenient access to shopping centers and dining establishments.
"""
]

user_prompt = """
generate at least 10 real estate listings following the guidelines
"""


In [27]:
# Here the Prompt is created via direct initiation (as oppose to using from_template)
# There are two way to initialize the variable, with Partial_variables or by formatting
# in first case (system_prompt), the partial prompt is used to pass the format instructions
# it also makes sense to do so, since we defined a pydantic object describing
# the format of the response

system_prompt = PromptTemplate(
    template=instructions,
      partial_variables={'output_format_instructions': format_instructions_homeListings,
                         'listing_example' : listing_example}

)

In [28]:
# Here is a second manner of passing the variables to after creating the prompt
# the count of listing will be passed at the time of creating the chain



user_query = PromptTemplate(
    template = user_prompt,
    input_variables=['listing_count'],
)

In [29]:
# user_prompt  = user_query.format(listing_count=NUMBER_OF_LISTINGS)
# user_prompt

In [30]:
print(format_instructions_homeListings)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"Listings": {"title": "Listings", "description": "list of real estate listings generated by the AI defined as singleHomeList model", "type": "array", "items": {"$ref": "#/definitions/singleHomeList"}}}, "required": ["Listings"], "definitions": {"singleHomeList": {"title": "singleHomeList", "type": "object", "properties": {"neighborhood": {"title": "Neighborhood", "description": "name of the neighborhood", "type": "string"}, "price": {"title": "Price", "description": "price of the home in US dollars", "type": "string"}, "bedroom

In [31]:
print(system_prompt)

input_variables=[] partial_variables={'output_format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"Listings": {"title": "Listings", "description": "list of real estate listings generated by the AI defined as singleHomeList model", "type": "array", "items": {"$ref": "#/definitions/singleHomeList"}}}, "required": ["Listings"], "definitions": {"singleHomeList": {"title": "singleHomeList", "type": "object", "properties": {"neighborhood": {"title": "Neighborhood", "description": "name of the neighborhood", "type": "string"}, "price": {"title": "Price", "

In [32]:
print(user_query)

input_variables=[] template='\ngenerate at least 10 real estate listings following the guidelines\n'


In [33]:
print(user_prompt)


generate at least 10 real estate listings following the guidelines



In [34]:
# prompt_template = PromptTemplate(
#     input_variables=['system','input'],
#     template = user_prompt
# )

In [35]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', "{system}"),
        ('human', "{input}")
    ]
)

In [36]:
prompt_template

ChatPromptTemplate(input_variables=['input', 'system'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['system'], template='{system}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [37]:


chain = get_chain(llm, prompt_template, parser_homeListings)


In [38]:
chain

ChatPromptTemplate(input_variables=['input', 'system'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['system'], template='{system}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7cf80a1ae200>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7cf80a1afac0>, model_name='gpt-3.5-turbo-1106', temperature=1.0, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=4000)
| PydanticOutputParser(pydantic_object=<class '__main__.HomeListings'>)

In [43]:
with get_openai_callback() as cb:
  realestate_listings = chain.invoke({'system'      : system_prompt,
                                      'input'       : user_prompt,
                                      })


In [44]:
print(cb)

Tokens Used: 3046
	Prompt Tokens: 1209
	Completion Tokens: 1837
Successful Requests: 1
Total Cost (USD): $0.004883


In [45]:
print(realestate_listings)

Listings=[singleHomeList(neighborhood='Green Oaks', price='$800,000', bedrooms=3, bathrooms=2, houseSize='2,000 sqft', description='Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 2,000 sqft, 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.', neighborhood_description='Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commutin

In [46]:
# display all the listing

for listing in realestate_listings.Listings:
  show_listing(listing)

Neighborhood:  Green Oaks
Price: $800,000
Bedroom: 3
Bathrooms: 2
House Size: 2,000 sqft
Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks.
This charming 2,000 sqft, 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar
panels and a well-insulated structure. Natural light floods the living spaces, highlighting the
beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead
to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace
sustainable living without compromising on style in this Green Oaks gem.
Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious
community with access to organic grocery stores, community gardens, and bike paths. Take a stroll
through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy
access to public transportation and bike lanes, commuting is a breeze.


Neighborhood:  

# Step 3: Storing Listings in a Vector Database

In [47]:
# In storing the listing the vector data, few attributes will be stored as metadata


# attributes that constitute the metadata in the vector database
keys = ['price', 'bedrooms', 'bathrooms','houseSize']


metadatas = []
ids = []
docs = []

dictList = []
idx = 0
for listing in realestate_listings.Listings:
  idx += 1

  d = listing.dict()
  d['listingId'] = 'listing-'+str(idx) # generate an id for the listing and store it in vectordb
  ids.append(d['listingId']) # create a list of the listing ids for storing as key in the vectordb

  metadatas.append({key: d[key] for key in d.keys() if key in keys  })
  docs.append(str(listing))

  dictList.append(d)


# # create the id for the listings
# ids = ["listing-"+str(i+1) for i in range(len(realestate_listings.Listings  ))]

print("metadatas\n {}".format(metadatas))
print("ids\n {}".format(ids))
print("docs\n {}".format(docs))
print("dictList\n {}".format(dictList))


metadatas
 [{'price': '$800,000', 'bedrooms': 3, 'bathrooms': 2, 'houseSize': '2,000 sqft'}, {'price': '$950,000', 'bedrooms': 3, 'bathrooms': 3, 'houseSize': '2,300 sqft'}, {'price': '$1,200,000', 'bedrooms': 4, 'bathrooms': 3, 'houseSize': '2,800 sqft'}, {'price': '$1,500,000', 'bedrooms': 5, 'bathrooms': 4, 'houseSize': '3,500 sqft'}, {'price': '$1,100,000', 'bedrooms': 4, 'bathrooms': 3, 'houseSize': '2,600 sqft'}, {'price': '$850,000', 'bedrooms': 3, 'bathrooms': 2, 'houseSize': '2,100 sqft'}, {'price': '$750,000', 'bedrooms': 3, 'bathrooms': 2, 'houseSize': '1,900 sqft'}, {'price': '$1,250,000', 'bedrooms': 4, 'bathrooms': 3, 'houseSize': '3,000 sqft'}, {'price': '$900,000', 'bedrooms': 4, 'bathrooms': 3, 'houseSize': '2,400 sqft'}, {'price': '$1,300,000', 'bedrooms': 5, 'bathrooms': 4, 'houseSize': '3,200 sqft'}]
ids
 ['listing-1', 'listing-2', 'listing-3', 'listing-4', 'listing-5', 'listing-6', 'listing-7', 'listing-8', 'listing-9', 'listing-10']
docs
 ["neighborhood='Green Oak

In [48]:
dictList[0].keys()

dict_keys(['neighborhood', 'price', 'bedrooms', 'bathrooms', 'houseSize', 'description', 'neighborhood_description', 'listingId'])

## Save the listing in a CSV file

In [49]:
with open(CSV_FILE, 'w') as csv_obj:
  csv_writer = csv.DictWriter(csv_obj, dictList[0].keys())
  csv_writer.writeheader()
  csv_writer.writerows(dictList)


## Store listing in the vector db

In [51]:



client = get_vector_db_client('./chromadb')

openai_ef = get_embedding_function()

collection = get_collection(client, 'home-listings', openai_ef)

In [52]:
collection.add(
    ids = ids,
    documents = docs,
    metadatas = metadatas,
)

# Step 4: Building the User Preference Interface

In [53]:
questions = ["How big do you want your house to be?",
             "What are 3 most important things for you in choosing this property?",
             "Which amenities would you like?",
             "Which transportation options are important to you?",
             "How urban do you want your neighborhood to be?",
            ]
answers = ["A comfortable 3 bedrooms house with a spacious kitchen and a cozy living room.",
           "A quiet neighborhood, good local schools, and convenient shopping options.",
           "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
           "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
           "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
          ]



In [54]:
# combine all the answers as a single string that is then used as as part of LLM prompt instructions
allAnswers = ''.join(answers)
print(allAnswers)

A comfortable 3 bedrooms house with a spacious kitchen and a cozy living room.A quiet neighborhood, good local schools, and convenient shopping options.A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.A balance between suburban tranquility and access to urban amenities like restaurants and theaters.


In [55]:
# define the questions and answers as history object.
# although both are hard coded but these very well can be captured interactively

history = ChatMessageHistory()

history.add_user_message(f""" you are an expert AI real estate agent and recommend a property to the user
based on their answer to the personal questions.
Ask user {len(questions)} questions
""")


for i in range(len(questions)):
  history.add_ai_message(questions[i])
  history.add_user_message(answers[i])

## Step 5: Searching Based on Preferences

### Semantic Search Implementation: Use the structured buyer preferences to perform a semantic search on the vector database, retrieving listings that most closely match the user's requirements.

In [56]:
format_user_answers = """
Parse the user responses as per for the format instructions. When information for some of the fields is not available in user reponses, update with string fields with ANY and
numeric fields with 0.
==================  START OF THE FORMATTING INSTRUCTIONS ==================
{format_instructions}
==================  END OF THE FORMATTING INSTRUCTIONS ==================

================== START OF THE USER RESPONSES ==================
{answers}
================== END OF THE USER RESPONSES ==================
"""

In [57]:
prompt = PromptTemplate(
    input_variables=['answers','format_instructions'],
    template = format_user_answers
)

In [58]:
prompt.format(answers=allAnswers, format_instructions=format_instructions_singleListing)

'\nParse the user responses as per for the format instructions. When information for some of the fields is not available in user reponses, update with string fields with ANY and\nnumeric fields with 0.\n==================  START OF THE FORMATTING INSTRUCTIONS ==================\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"neighborhood": {"title": "Neighborhood", "description": "name of the neighborhood", "type": "string"}, "price": {"title": "Price", "description": "price of the home in US dollars", "type": "string"}, "bedrooms": {"title": "Bedrooms", "descriptio

In [60]:
chain = get_chain(llm, prompt, parser_singleListing)


In [61]:
with get_openai_callback() as cb:
  formatted_answers = chain.invoke({'answers' : allAnswers,
                                    'format_instructions' : format_instructions_singleListing
                                    })

In [62]:
print(cb)

Tokens Used: 665
	Prompt Tokens: 526
	Completion Tokens: 139
Successful Requests: 1
Total Cost (USD): $0.000804


In [63]:
print(formatted_answers)

neighborhood='ANY' price='ANY' bedrooms=3 bathrooms=0 houseSize='ANY' description='A comfortable 3 bedrooms house with a spacious kitchen and a cozy living room. A quiet neighborhood, good local schools, and convenient shopping options. A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. A balance between suburban tranquility and access to urban amenities like restaurants and theaters.' neighborhood_description='ANY'


In [65]:
# search the vector database, using the formatted/parsed answers parsed above
semantic_search_result = collection.query(query_texts=str(formatted_answers),
                                          n_results=4,
                                          )

semantic_search_result

{'ids': [['listing-5', 'listing-7', 'listing-2', 'listing-3']],
 'distances': [[0.7177660878567762,
   0.7350961648746905,
   0.7560238771726391,
   0.7583421921939868]],
 'metadatas': [[{'bathrooms': 3,
    'bedrooms': 4,
    'houseSize': '2,600 sqft',
    'price': '$1,100,000'},
   {'bathrooms': 2,
    'bedrooms': 3,
    'houseSize': '1,900 sqft',
    'price': '$750,000'},
   {'bathrooms': 3,
    'bedrooms': 3,
    'houseSize': '2,300 sqft',
    'price': '$950,000'},
   {'bathrooms': 3,
    'bedrooms': 4,
    'houseSize': '2,800 sqft',
    'price': '$1,200,000'}]],
 'embeddings': None,
 'documents': [["neighborhood='Mountain View' price='$1,100,000' bedrooms=4 bathrooms=3 houseSize='2,600 sqft' description='Discover the perfect blend of sophistication and comfort in this exquisite 4-bedroom, 3-bathroom residence, spanning 2,600 sqft. The modern design, high ceilings, and abundant natural light create a welcoming ambiance that flows seamlessly into the private outdoor living spaces. E

### we can choose to return a single result from the vector database, but here we returned multiple results and utilize LLM to get us the best listing that matches the user's preferences

In [66]:
result_documents = semantic_search_result.get('documents')
result_ids = semantic_search_result.get('ids')


In [67]:
result_ids

[['listing-5', 'listing-7', 'listing-2', 'listing-3']]

In [68]:
llm_query = """
Choose the one best listing from the following context that best match based on the user answers, reformat the selected as per the format instructions.

================== START OF THE FORMATTING INSTRUCTIONS ==================
{format_instructions}
================== END OF THE FORMATTING INSTRUCTIONS ==================

============ START OF THE CONTEXT ============
{context}
============ END OF THE CONTEXT ============

============ START OF THE USER ANSWERS ============
{answers}
============ END OF THE USER ANSWERS ============

"""

In [69]:
llm_query

'\nChoose the one best listing from the following context that best match based on the user answers, reformat the selected as per the format instructions. \n\n================== START OF THE FORMATTING INSTRUCTIONS ==================\n{format_instructions}\n================== END OF THE FORMATTING INSTRUCTIONS ==================\n\n============ START OF THE CONTEXT ============\n{context}\n============ END OF THE CONTEXT ============\n\n============ START OF THE USER ANSWERS ============\n{answers}\n============ END OF THE USER ANSWERS ============\n\n'

In [70]:
llm_prompt = PromptTemplate(
    input_variables=['context',  'answers', 'format_instructions'],
    template = llm_query
    )

In [71]:
chain = get_chain(llm, llm_prompt, parser_singleListing)


In [72]:
with get_openai_callback() as cb:
  best_listing = chain.invoke({'context' : result_documents,
                               'answers' : allAnswers,
                               'format_instructions' : format_instructions_singleListing
                            })

In [73]:
show_listing(best_listing)
# best_listing

Neighborhood:  Hillside Heights
Price: $750,000
Bedroom: 3
Bathrooms: 2
House Size: 1,900 sqft
Description: Welcome to this enchanting 3-bedroom, 2-bathroom hillside retreat offering
1,900 sqft of cozy living space. The expansive outdoor deck provides a scenic vantage point to
admire the breathtaking sunsets, while the open-concept floor plan invites relaxation and
entertainment. Experience the serene charm of Hillside Heights in this delightful residence.
Neighborhood Description: Hillside Heights offers a peaceful escape from the hustle and
bustle, with lush greenery, nature trails, and exclusive hillside views. Residents find solace in
the tranquility of the neighborhood while staying conveniently close to local amenities and scenic
overlooks.




### lets have LLM explain the reason how the choosen listing best matches the user preferences

In [74]:
best_listing_reason = """
Highlight the featues in the  listing that matches the user answers.
============== START OF THE LISTING ================
{listing}
============== END OF THE LISTING ================

============== START OF THE USER ANSWERS ================
{answers}
============== END OF THE USER ANSWERS ================

"""


In [75]:
prompt_augment_listing = PromptTemplate(
    input_variables=['listing', 'answers'],
    template = best_listing_reason
)

In [76]:
chain = get_chain(llm, prompt_augment_listing, None)


In [77]:
with get_openai_callback() as cb:
  recommendation = chain.invoke({'listing' : best_listing.dict(),
                            'answers' : allAnswers
                            })


In [78]:
print(recommendation.content)

- 3 bedrooms
- 2 bathrooms
- Cozy living space
- Peaceful neighborhood 
- Convenient access to local amenities


In [79]:
print(style.UNDERLINE + style.BOLD)
print ("\nUser preference ")
print(style.END)
print(textwrap.fill(allAnswers, 100))

print(style.UNDERLINE + style.BOLD)
print ("\nselected Listing based on the user preferences")
print(style.END)

show_listing(best_listing)
# for k,v in best_listing.dict().items():
#   print(textwrap.fill(f"{k} : {v}", 100 ))

print(style.UNDERLINE + style.BOLD)
print ("\nRecommendation explanation")
print(style.END)
print(recommendation.content)





User preference 

A comfortable 3 bedrooms house with a spacious kitchen and a cozy living room.A quiet neighborhood,
good local schools, and convenient shopping options.A backyard for gardening, a two-car garage, and
a modern, energy-efficient heating system.Easy access to a reliable bus line, proximity to a major
highway, and bike-friendly roads.A balance between suburban tranquility and access to urban
amenities like restaurants and theaters.


selected Listing based on the user preferences

Neighborhood:  Hillside Heights
Price: $750,000
Bedroom: 3
Bathrooms: 2
House Size: 1,900 sqft
Description: Welcome to this enchanting 3-bedroom, 2-bathroom hillside retreat offering
1,900 sqft of cozy living space. The expansive outdoor deck provides a scenic vantage point to
admire the breathtaking sunsets, while the open-concept floor plan invites relaxation and
entertainment. Experience the serene charm of Hillside Heights in this delightful residence.
Neighborhood Description: Hillside He

# Step 6: Personalizing Listing Descriptions

In [80]:
prompt_Listpersonalized_instructions = """
========== START OF USER PREFERENCES QUESTIONS AND ANSWERS ===========
{history}
========== END OF USER PREFERENCES QUESTIONS AND ANSWERS  ============

========== START REAL ESTATE LISTING ===========
{listingDescription}
========== END REAL ESTATE LISTING ============

PERSONALIZATION INSTRUCTIONS THAT MUST BE FOLLOWED:
Craft a personalized description of the real estate listing based on the listing and highlight the features that match the user preferences
and make it captivating and engaging. Donot alter the facts.
"""

prompt_final_recommendation = """
Now that the AI has personalized all the real estate listings, AI will recommend human top three listing that human
will like the most and describes the reason why these best meets the human preferences.

"""


In [81]:
personalized_listings = []

for i in range(collection.count()):
  listing = (collection.get(include=['metadatas','documents'],
                            offset=i,
                            limit=1
                            ))


  listingId = listing['ids']
  listingMetadatas = listing['metadatas']
  listingDescription = listing['documents']

  prompt_template_2 = PromptTemplate(
      input_variables=[listingId, listingMetadatas, listingDescription, history ],
      template = prompt_Listpersonalized_instructions
  )

  personalized_list = {}
  personalized_list['listing-id'] = listingId

  # chain = prompt_template_2 | llm
  chain = get_chain(llm, prompt_template_2, None)

  personalized_list['personalized_description'] = chain.invoke({'listingDescription' : listingDescription,
                                                                'history' : history,
                                                                })

  personalized_listings.append(personalized_list)




print(len(personalized_listings))

10


In [82]:
realestate_listings = csv.DictReader(open(CSV_FILE))

for l in realestate_listings:
  singleListObj = singleHomeList(**l)
  show_listing (singleListObj)

  for p in personalized_listings:
    if p['listing-id'][0] == l['listingId']:
      print(style.BOLD+style.BLUE+'Personalized Description'+style.END)
      print(textwrap.fill(p['personalized_description'].content),100)
      print("\n")
      break



Neighborhood:  Green Oaks
Price: $800,000
Bedroom: 3
Bathrooms: 2
House Size: 2,000 sqft
Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks.
This charming 2,000 sqft, 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar
panels and a well-insulated structure. Natural light floods the living spaces, highlighting the
beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead
to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace
sustainable living without compromising on style in this Green Oaks gem.
Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious
community with access to organic grocery stores, community gardens, and bike paths. Take a stroll
through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy
access to public transportation and bike lanes, commuting is a breeze.


Personalized De

# THE END